In [2]:
!curl -O https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  3832  100  3832    0     0   5855      0 --:--:-- --:--:-- --:--:--  5895
100  3832  100  3832    0     0   5854      0 --:--:-- --:--:-- --:--:--  5886


In [14]:
import pandas as pd
import minsearch

## Ingestion

In [37]:
df = pd.read_csv('../data/data.csv')

In [38]:
df.columns

Index(['id', 'exercise_name', 'type_of_activity', 'type_of_equipment',
       'body_part', 'type', 'muscle_groups_activated', 'instructions'],
      dtype='object')

In [39]:
df

,id,exercise_name,type_of_activity,type_of_equipment,body_part,type,muscle_groups_activated,instructions
0,0,Push-Ups,Strength,Bodyweight,Upper Body,Push,"Pectorals, Triceps, Deltoids",Start in a high plank position with your hands...
1,1,Squats,Strength,Bodyweight,Lower Body,Push,"Quadriceps, Glutes, Hamstrings",Stand with feet shoulder-width apart. Lower yo...
2,2,Plank,Strength/Mobility,Bodyweight,Core,Hold,"Rectus Abdominis, Transverse Abdominis",Start in a forearm plank position with your el...
3,3,Deadlift,Strength,Barbell,Lower Body,Pull,"Glutes, Hamstrings, Lower Back","Stand with feet hip-width apart, barbell in fr..."
4,4,Bicep Curls,Strength,Dumbbells,Upper Body,Pull,"Biceps, Forearms","Stand with a dumbbell in each hand, arms fully..."
...,...,...,...,...,...,...,...,...
202,202,Incline Dumbbell Row,Strength,Dumbbells,Upper Body,Pull,"Latissimus Dorsi, Biceps",Lie face down on an incline bench with a dumbb...
203,203,Machine Lat Pulldown,Strength,Machine,Upper Body,Pull,"Latissimus Dorsi, Biceps",Sit at a lat pulldown machine with a wide grip...
204,204,One-Arm Cable Row,Strength,Cable Machine,Upper Body,Pull,"Latissimus Dorsi, Biceps",Stand facing a cable machine with the handle a...
205,205,Face Pull,Strength,Cable Machine,Upper Body,Pull,"Rear Deltoids, Trapezius, Rhomboids",Attach a rope handle to a high cable pulley. P...


In [45]:
documents = df.to_dict(orient='records')

In [46]:
documents

[{'id': 0,
  'exercise_name': 'Push-Ups',
  'type_of_activity': 'Strength',
  'type_of_equipment': 'Bodyweight',
  'body_part': 'Upper Body',
  'type': 'Push',
  'muscle_groups_activated': 'Pectorals, Triceps, Deltoids',
  'instructions': 'Start in a high plank position with your hands under your shoulders. Lower your body until your chest nearly touches the floor. Push back up to the starting position.'},
 {'id': 1,
  'exercise_name': 'Squats',
  'type_of_activity': 'Strength',
  'type_of_equipment': 'Bodyweight',
  'body_part': 'Lower Body',
  'type': 'Push',
  'muscle_groups_activated': 'Quadriceps, Glutes, Hamstrings',
  'instructions': 'Stand with feet shoulder-width apart. Lower your body as if sitting back into a chair, keeping your chest up. Return to standing.'},
 {'id': 2,
  'exercise_name': 'Plank',
  'type_of_activity': 'Strength/Mobility',
  'type_of_equipment': 'Bodyweight',
  'body_part': 'Core',
  'type': 'Hold',
  'muscle_groups_activated': 'Rectus Abdominis, Transvers

In [47]:
index = minsearch.Index(
    text_fields = ['exercise_name', 'type_of_activity', 'type_of_equipment', 'body_part',
       'type', 'muscle_groups_activated', 'instructions'],
    keyword_fields = ['id']
)

In [48]:
index.fit(documents)

In [21]:
query = "give me leg exercises for hamstrings"

In [49]:
index.search(query, num_results=10)

[{'id': 118,
  'exercise_name': 'Leg Curl',
  'type_of_activity': 'Strength',
  'type_of_equipment': 'Machine',
  'body_part': 'Lower Body',
  'type': 'Pull',
  'muscle_groups_activated': 'Hamstrings',
  'instructions': 'Lie face down on a leg curl machine and curl the pad towards your glutes, then return to the starting position.'},
 {'id': 95,
  'exercise_name': 'Machine Leg Curl',
  'type_of_activity': 'Strength',
  'type_of_equipment': 'Machine',
  'body_part': 'Lower Body',
  'type': 'Pull',
  'muscle_groups_activated': 'Hamstrings',
  'instructions': 'Sit on a leg curl machine and curl the pad down towards your glutes, then return to the starting position.'},
 {'id': 109,
  'exercise_name': 'Seated Leg Curl',
  'type_of_activity': 'Strength',
  'type_of_equipment': 'Machine',
  'body_part': 'Lower Body',
  'type': 'Pull',
  'muscle_groups_activated': 'Hamstrings',
  'instructions': 'Sit on a leg curl machine and curl the pad down towards your glutes, then return to the starting p

## RAG Flow

In [23]:
from openai import OpenAI

In [24]:
client = OpenAI()

In [27]:
response = client.chat.completions.create(
    model = 'gpt-4o-mini', 
    messages = [{'role':'user', 'content':query}]
)

response.choices[0].message.content # It Gave the General response without any context knowledge

"Strengthening your hamstrings is important for overall leg health and functionality. Here are some effective exercises specifically targeting the hamstrings:\n\n1. **Deadlifts**:\n   - **Traditional Deadlift**: Stand with feet hip-width apart, hold a barbell or dumbbells in front of you, and hinge at the hips while lowering the weights down the front of your legs. Keep your back straight and engage your hamstrings as you return to a standing position.\n   - **Romanian Deadlift**: Similar to the traditional deadlift, but keep a slight bend in the knees and focus more on hinging at the hips, lowering the weights until you feel a stretch in your hamstrings.\n\n2. **Leg Curls**:\n   - **Machine Leg Curl**: Sit or lie down on a leg curl machine with your ankles under the padded lever. Curl the weight towards your glutes, squeezing the hamstrings at the top.\n   - **Standing or Lying Leg Curl with Resistance Band**: Attach a resistance band to a sturdy object and wrap it around your ankle. 

In [50]:
def search(query):
    # search function (minsearch)
    boost = {} #to give the imporatnace to each keywords by default it's 1.
    
    results = index.search(
             query=query,
             filter_dict={}, #to filter only DE results
             boost_dict = boost,
             num_results = 5
            )
    
    return results

In [29]:
prompt_template = """
    You are a fitness coach instructor. Answer the QUESTION based on the CONTEXT from our exercises database. 
    Use only facts form the CONTEXT while answering the QUESTION.
    If the CONTEXT doesn't contain the answer, output NONE.

    QUESTION : {question}

    CONTEXT : {context}
    """.strip()

entry_template = """
'exercise_name': {exercise_name}
'type_of_activity': {type_of_activity}
'type_of_equipment': {type_of_equipment}
'body_part': {body_part}
'type': {type}
'muscle_groups_activated': {muscle_groups_activated}
'instructions': {instructions}
""".strip()


def build_prompt(query, search_result):
    # function to build the prompt
    
    context = ""
    for doc in search_result:
        context = context + entry_template.format(**doc) + "\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [33]:
def llm(prompt):
    response = client.chat.completions.create(
        model = 'gpt-4o-mini', 
        messages = [{'role':'user', 'content':prompt}]
    )
    return response.choices[0].message.content

In [34]:
def rag(query):
    search_result = search(query)
    prompt = build_prompt(query, search_result)
    answer = llm(prompt)

    return answer

In [54]:
answer = rag(query)
print(answer)

Here are leg exercises for hamstrings:

1. **Leg Curl**  
   - Type of Activity: Strength  
   - Type of Equipment: Machine  
   - Instructions: Lie face down on a leg curl machine and curl the pad towards your glutes, then return to the starting position.

2. **Machine Leg Curl**  
   - Type of Activity: Strength  
   - Type of Equipment: Machine  
   - Instructions: Sit on a leg curl machine and curl the pad down towards your glutes, then return to the starting position.

3. **Seated Leg Curl**  
   - Type of Activity: Strength  
   - Type of Equipment: Machine  
   - Instructions: Sit on a leg curl machine and curl the pad down towards your glutes, then return to the starting position.

4. **Lying Leg Curl**  
   - Type of Activity: Strength  
   - Type of Equipment: Machine  
   - Instructions: Lie face down on a leg curl machine and curl the pad towards your glutes, then return to the starting position.

5. **Prone Leg Curl**  
   - Type of Activity: Strength  
   - Type of Equipm

In [36]:
answer = rag('I want some core exercises to help the backbending')
print(answer)

Here are some core exercises that can help with backbending:

1. **Landmine Twist**  
   - **Type of Activity**: Strength  
   - **Equipment**: Barbell  
   - **Muscle Groups Activated**: Obliques, Rectus Abdominis  
   - **Instructions**: Hold the end of a barbell anchored in a landmine attachment. Rotate the bar side to side while keeping your core tight.

2. **Ab Wheel Rollout**  
   - **Type of Activity**: Strength  
   - **Equipment**: Ab Wheel  
   - **Muscle Groups Activated**: Rectus Abdominis, Hip Flexors  
   - **Instructions**: Kneel on the floor with an ab wheel in front of you. Roll the wheel forward while keeping your core tight, then roll back.

3. **Spider Plank**  
   - **Type of Activity**: Strength  
   - **Equipment**: Bodyweight  
   - **Muscle Groups Activated**: Rectus Abdominis, Obliques  
   - **Instructions**: Start in a forearm plank position. Bring one knee to the outside of your elbow, then return to the starting position and repeat on the other side.

4. *

In [55]:
answer = rag('I want some exercises to help me with the pull ups')
print(answer)

Here are some exercises to help you with pull-ups:

1. **Pull-Ups**
   - **Type of Activity:** Strength
   - **Type of Equipment:** Bodyweight
   - **Body Part:** Upper Body
   - **Muscle Groups Activated:** Latissimus Dorsi, Biceps
   - **Instructions:** Grasp a pull-up bar with an overhand grip. Pull your body up until your chin is above the bar, then lower back down.

2. **Chin-Ups**
   - **Type of Activity:** Strength
   - **Type of Equipment:** Bodyweight
   - **Body Part:** Upper Body
   - **Muscle Groups Activated:** Latissimus Dorsi, Biceps
   - **Instructions:** Grip a pull-up bar with an underhand grip. Pull your body up until your chin is above the bar, then lower back down.

These exercises will help strengthen the muscles used in pull-ups.


In [57]:
question = "Which body part primarily benefits from doing push-ups?"
answer = rag(question)
print(answer)

The body part that primarily benefits from doing push-ups is the Upper Body.


## Retrieval Evaluation:

In [58]:
df_questions = pd.read_csv('../data/ground-truth-retrieval.csv')

In [59]:
df_questions

,id,question
0,0,What is the starting position for doing push-ups?
1,0,Which muscle groups are activated during push-...
2,0,How do you know when to push back up while doi...
3,0,Do you need any equipment to perform push-ups?
4,0,What part of the body do push-ups primarily ta...
...,...,...
1030,206,What muscles are primarily worked during the D...
1031,206,What type of exercise is the Dumbbell Bench Pr...
1032,206,Can the Dumbbell Bench Press be performed with...
1033,206,What body part does the Dumbbell Bench Press f...


In [60]:
ground_truth = df_questions.to_dict(orient='records')

In [61]:
ground_truth[0]

{'id': 0, 'question': 'What is the starting position for doing push-ups?'}

In [62]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [63]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [64]:
def minsearch_search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [66]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['id']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [67]:
from tqdm.auto import tqdm

In [68]:
evaluate(ground_truth, lambda q: minsearch_search(q['question']))

  0%|          | 0/1035 [00:00<?, ?it/s]

{'hit_rate': 0.9478260869565217, 'mrr': 0.822744038033893}

## Finding the best parameters


In [70]:
df_validation = df_questions[:100]
df_test = df_questions[100:]

In [71]:
import random

def simple_optimize(param_ranges, objective_function, n_iterations=10):
    best_params = None
    best_score = float('-inf')  # Assuming we're minimizing. Use float('-inf') if maximizing.

    for _ in range(n_iterations):
        # Generate random parameters
        current_params = {}
        for param, (min_val, max_val) in param_ranges.items():
            if isinstance(min_val, int) and isinstance(max_val, int):
                current_params[param] = random.randint(min_val, max_val)
            else:
                current_params[param] = random.uniform(min_val, max_val)
        
        # Evaluate the objective function
        current_score = objective_function(current_params)
        
        # Update best if current is better
        if current_score > best_score:  # Change to > if maximizing
            best_score = current_score
            best_params = current_params
    
    return best_params, best_score

In [72]:
gt_val = df_validation.to_dict(orient='records')

In [73]:
def minsearch_search(query, boost=None):
    if boost is None:
        boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [74]:
param_ranges = {
    'exercise_name': (0.0, 3.0),
    'type_of_activity': (0.0, 3.0),
    'type_of_equipment': (0.0, 3.0),
    'body_part': (0.0, 3.0),
    'type': (0.0, 3.0),
    'muscle_groups_activated': (0.0, 3.0),
    'instructions': (0.0, 3.0),
}

def objective(boost_params):
    def search_function(q):
        return minsearch_search(q['question'], boost_params)

    results = evaluate(gt_val, search_function)
    return results['mrr']

In [75]:
simple_optimize(param_ranges, objective, n_iterations=20)


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

({'exercise_name': 1.1162887216689357,
  'type_of_activity': 1.2053964373475001,
  'type_of_equipment': 0.0157474211791101,
  'body_part': 0.5117172999420851,
  'type': 1.9562913342437052,
  'muscle_groups_activated': 2.7508697687269983,
  'instructions': 0.27669361047746277},
 0.847)

In [76]:
def minsearch_improved(query):
    boost = {
        'exercise_name': 2.11,
        'type_of_activity': 1.46,
        'type_of_equipment': 0.65,
        'body_part': 2.65,
        'type': 1.31,
        'muscle_groups_activated': 2.54,
        'instructions': 0.74
    }

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

evaluate(ground_truth, lambda q: minsearch_improved(q['question']))

  0%|          | 0/1035 [00:00<?, ?it/s]

{'hit_rate': 0.9468599033816425, 'mrr': 0.9029733149298367}

## RAG Evaluation

In [77]:
prompt2_template = """
You are an expert evaluator for a Retrieval-Augmented Generation (RAG) system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: {question}
Generated Answer: {answer_llm}

Please analyze the content and context of the generated answer in relation to the question
and provide your evaluation in parsable JSON without using code blocks:

{{
  "Relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "Explanation": "[Provide a brief explanation for your evaluation]"
}}
""".strip()

In [78]:
len(ground_truth)


1035

In [81]:
record = ground_truth[0]
record

{'id': 0, 'question': 'What is the starting position for doing push-ups?'}

In [84]:
record = ground_truth[0]
question = record['question']
answer_llm = rag(question)

In [85]:
print(answer_llm)

The starting position for doing push-ups is to start in a high plank position with your hands under your shoulders.


In [87]:
prompt = prompt2_template.format(question=question, answer_llm=answer_llm)
print(prompt)

You are an expert evaluator for a Retrieval-Augmented Generation (RAG) system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: What is the starting position for doing push-ups?
Generated Answer: The starting position for doing push-ups is to start in a high plank position with your hands under your shoulders.

Please analyze the content and context of the generated answer in relation to the question
and provide your evaluation in parsable JSON without using code blocks:

{
  "Relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "Explanation": "[Provide a brief explanation for your evaluation]"
}


In [89]:
print(llm(prompt))

{
  "Relevance": "RELEVANT",
  "Explanation": "The generated answer accurately describes the starting position for doing push-ups, specifically mentioning the high plank position and the placement of hands, which directly addresses the question."
}


In [90]:
evaluations = {}


In [91]:
import json

In [98]:
df_sample = df_questions.sample(n=200, random_state=1)
sample = df_sample.to_dict(orient='records')


In [99]:
evaluations = []

for record in tqdm(sample):
    question = record['question']
    answer_llm = rag(question) 

    prompt = prompt2_template.format(
        question=question,
        answer_llm=answer_llm
    )

    evaluation = llm(prompt)
    evaluation = json.loads(evaluation)

    evaluations.append((record, answer_llm, evaluation))
    

  0%|          | 0/200 [00:00<?, ?it/s]

In [100]:
evaluations

[({'id': 171,
   'question': 'What is the primary muscle group activated during the Banded Pull-Up?'},
  'The primary muscle group activated during the Banded Pull-Up is the Latissimus Dorsi.',
  {'Relevance': 'RELEVANT',
   'Explanation': 'The generated answer correctly identifies the primary muscle group activated during the Banded Pull-Up, which directly addresses the question asked.'}),
 ({'id': 115,
   'question': 'Can jumping squats be performed without any equipment?'},
  'Yes, jumping squats can be performed without any equipment, as they use bodyweight.',
  {'Relevance': 'RELEVANT',
   'Explanation': 'The generated answer directly addresses the question by confirming that jumping squats can be performed without any equipment and explains that they utilize bodyweight, which is pertinent information.'}),
 ({'id': 53,
   'question': 'Can you explain the proper technique for executing a Dumbbell Lateral Raise?'},
  'To execute a Dumbbell Lateral Raise properly, stand with a dumbbe

In [101]:
df_eval = pd.DataFrame(evaluations, columns=['record', 'answer', 'evaluation'])
df_eval

,record,answer,evaluation
0,"{'id': 171, 'question': 'What is the primary m...",The primary muscle group activated during the ...,"{'Relevance': 'RELEVANT', 'Explanation': 'The ..."
1,"{'id': 115, 'question': 'Can jumping squats be...","Yes, jumping squats can be performed without a...","{'Relevance': 'RELEVANT', 'Explanation': 'The ..."
2,"{'id': 53, 'question': 'Can you explain the pr...","To execute a Dumbbell Lateral Raise properly, ...","{'Relevance': 'RELEVANT', 'Explanation': 'The ..."
3,"{'id': 198, 'question': 'How do I ensure my ar...",To ensure your arms are fully extended during ...,"{'Relevance': 'RELEVANT', 'Explanation': 'The ..."
4,"{'id': 19, 'question': 'What part of the body ...",The part of the body primarily engaged in Gobl...,"{'Relevance': 'PARTLY_RELEVANT', 'Explanation'..."
...,...,...,...
195,"{'id': 83, 'question': 'What specific body par...","The Pendlay Row targets the Upper Body, specif...","{'Relevance': 'PARTLY_RELEVANT', 'Explanation'..."
196,"{'id': 94, 'question': 'How does the TRX varia...",The TRX variation differs from traditional pus...,"{'Relevance': 'RELEVANT', 'Explanation': 'The ..."
197,"{'id': 123, 'question': 'Can beginners safely ...",NONE,"{'Relevance': 'NON_RELEVANT', 'Explanation': '..."
198,"{'id': 76, 'question': 'How should I position ...",Lie on your back with legs extended. Lift your...,"{'Relevance': 'RELEVANT', 'Explanation': 'The ..."


In [102]:
df_eval['id'] = df_eval.record.apply(lambda d: d['id'])
df_eval['question'] = df_eval.record.apply(lambda d: d['question'])

df_eval['relevance'] = df_eval.evaluation.apply(lambda d: d['Relevance'])
df_eval['explanation'] = df_eval.evaluation.apply(lambda d: d['Explanation'])

In [103]:
del df_eval['record']
del df_eval['evaluation']

In [104]:
df_eval.relevance.value_counts(normalize=True)


relevance
RELEVANT           0.695
PARTLY_RELEVANT    0.185
NON_RELEVANT       0.120
Name: proportion, dtype: float64

In [105]:
df_eval.relevance.value_counts()


relevance
RELEVANT           139
PARTLY_RELEVANT     37
NON_RELEVANT        24
Name: count, dtype: int64

In [106]:
df_eval.to_csv('../data/rag-eval-gpt-4o-mini.csv', index=False)


In [107]:
df_eval[df_eval.relevance == 'NON_RELEVANT']


,answer,id,question,relevance,explanation
19,The One-Arm Dumbbell Snatch is considered a pu...,190,Is the One-Arm Dumbbell Snatch considered a pu...,NON_RELEVANT,The generated answer incorrectly categorizes t...
22,NONE,30,Can calf raises target any other body parts be...,NON_RELEVANT,The generated answer 'NONE' does not provide a...
49,NONE,36,What is the correct position to start the Incl...,NON_RELEVANT,The generated answer 'NONE' does not provide a...
52,The Cable Lateral Raise is a push exercise.,100,Is the Cable Lateral Raise a push or pull exer...,NON_RELEVANT,The generated answer incorrectly categorizes t...
57,NONE,99,What is the appropriate weight for the dumbbel...,NON_RELEVANT,The generated answer 'NONE' does not address t...
64,NONE,54,How long should I hold the Wall Sit position f...,NON_RELEVANT,The generated answer 'NONE' does not provide a...
67,NONE,107,Can you explain the correct stance for perform...,NON_RELEVANT,"The generated answer is 'NONE', which fails to..."
69,NONE,17,What is the primary body position to start per...,NON_RELEVANT,The generated answer 'NONE' does not provide a...
76,NONE,187,Is the Reverse Grip Lat Pulldown suitable for ...,NON_RELEVANT,The generated answer 'NONE' does not provide a...
79,NONE,99,How do I avoid injury while doing the Dumbbell...,NON_RELEVANT,The generated answer 'NONE' does not provide a...
